빅쿼리에서 데이터를 정제하여 쓸 수 있는 데이터들만 남깁니다. 

In [ ]:
from google.colab import drive
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os

# ==============================================================================
# 1. 환경 설정 및 구글 인증
# ==============================================================================
# 1) 구글 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

# 2) BigQuery 인증 (팝업 로그인 필요)
print("🔑 Google Cloud 인증 중...")
auth.authenticate_user()
PROJECT_ID = 'YOUR_PROJECT_ID_HERE'
client = bigquery.Client(project=PROJECT_ID)

# 경로 설정
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/'
SAVE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'
os.makedirs(SAVE_DIR, exist_ok=True)

# 파일 경로 정의
PATH_TRIAGE = BASE_DIR + 'triage.csv'
PATH_EDSTAYS = BASE_DIR + 'edstays.csv'
PATH_PATIENTS = BASE_DIR + 'patients.csv'
PATH_META = BASE_DIR + 'mimic-cxr-2.0.0-metadata.csv'
PATH_CHEXPERT = BASE_DIR + 'mimic-cxr-2.0.0-chexpert.csv'

print("🚀 [Step 1] 데이터 정제 및 BigQuery 연동 프로세스 시작...")

try:
    # ==============================================================================
    # 2. 기본 데이터 로드
    # ==============================================================================
    print("\n📥 1. CSV 파일 로딩 중...")
    df_triage = pd.read_csv(PATH_TRIAGE)
    df_edstays = pd.read_csv(PATH_EDSTAYS, usecols=['subject_id', 'stay_id', 'intime', 'outtime'])
    df_patients = pd.read_csv(PATH_PATIENTS, usecols=['subject_id', 'gender', 'anchor_age'])
    df_meta = pd.read_csv(PATH_META, usecols=['dicom_id', 'subject_id', 'study_id', 'StudyDate', 'StudyTime', 'ViewPosition'])
    df_chexpert = pd.read_csv(PATH_CHEXPERT)

    print(f"   - Triage: {len(df_triage)} rows")

    # ==============================================================================
    # 3. 응급실 데이터 병합 (Base DataFrame 생성)
    # ==============================================================================
    print("\n⏳ 2. 응급실 데이터 병합 및 시간 포맷 변환 중...")

    # 여기서 df_ed_full이 최초 생성됩니다.
    df_ed_full = pd.merge(df_triage, df_edstays, on=['subject_id', 'stay_id'], how='inner')

    df_ed_full['intime'] = pd.to_datetime(df_ed_full['intime'])
    df_ed_full['outtime'] = pd.to_datetime(df_ed_full['outtime'])

    # ==============================================================================
    # [NEW] 3.5. BigQuery Lab 데이터 (WBC, BNP) 병합 로직 삽입
    # ==============================================================================
    print("\n🚀 2.5. BigQuery에서 Lab 데이터(WBC, BNP) 추출 중...")

    query = """
    SELECT subject_id, charttime, itemid, valuenum
    FROM `physionet-data.mimiciv_3_1_hosp.labevents`
    WHERE itemid IN (51301, 50963) AND valuenum IS NOT NULL
    """
    df_lab = client.query(query).to_dataframe()
    df_lab['charttime'] = pd.to_datetime(df_lab['charttime'])

    # Lab 데이터 병합을 위한 임시 테이블
    df_ed_subset = df_ed_full[['subject_id', 'stay_id', 'intime']].copy()
    df_merged_lab = pd.merge(df_ed_subset, df_lab, on='subject_id', how='inner')

    # 시간 필터링 (±24시간)
    df_merged_lab['time_diff'] = (df_merged_lab['charttime'] - df_merged_lab['intime']).abs()
    df_merged_lab = df_merged_lab[df_merged_lab['time_diff'] <= pd.Timedelta(hours=24)].copy()

    # 중복 제거 (가장 가까운 시간 선택)
    df_merged_lab.sort_values('time_diff', inplace=True)
    df_merged_lab.drop_duplicates(subset=['stay_id', 'itemid'], keep='first', inplace=True)

    # Pivot (Long -> Wide)
    df_lab_pivot = df_merged_lab.pivot(index='stay_id', columns='itemid', values='valuenum')
    df_lab_pivot.rename(columns={51301: 'WBC', 50963: 'BNP'}, inplace=True)

    # 메인 데이터(df_ed_full)에 병합
    df_ed_full = pd.merge(df_ed_full, df_lab_pivot, on='stay_id', how='left')

    # 결측치 -1 처리
    for col in ['WBC', 'BNP']:
        if col not in df_ed_full.columns:
            df_ed_full[col] = -1
        else:
            df_ed_full[col] = df_ed_full[col].fillna(-1)

    print(f"   ✅ Lab 데이터 병합 완료! (WBC 결측 비율: {(df_ed_full['WBC'] == -1).mean()*100:.1f}%)")

    # ==============================================================================
    # 4. 이미지 시점 매칭 및 정제 (기존 로직 계속)
    # ==============================================================================
    # CXR 시간 파싱
    df_meta['StudyTime'] = df_meta['StudyTime'].astype(str).str.split('.').str[0].str.zfill(6)
    df_meta['StudyDateTime'] = pd.to_datetime(
        df_meta['StudyDate'].astype(str) + ' ' + df_meta['StudyTime'],
        format='%Y%m%d %H%M%S', errors='coerce'
    )

    print("\n🔗 3. 엑스레이 vs 응급실 체류 시간 대조...")
    # 이제 df_ed_full에는 WBC, BNP가 포함되어 있으므로 그대로 병합됩니다.
    df_merged = pd.merge(df_ed_full, df_meta, on='subject_id', how='inner')

    time_mask = (
        (df_merged['StudyDateTime'] >= df_merged['intime'] - pd.Timedelta(hours=2)) &
        (df_merged['StudyDateTime'] <= df_merged['outtime'] + pd.Timedelta(hours=2))
    )
    df_matched = df_merged[time_mask].copy()

    # 데이터 품질 관리
    print("\n🧹 4. 데이터 품질 관리 (Cleaning)...")
    df_matched = df_matched[df_matched['ViewPosition'].isin(['AP', 'PA'])].copy()

    # 바이탈 결측 제거 (WBC, BNP는 -1로 채웠으므로 여기서 삭제되지 않습니다)
    vital_cols = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
    df_matched.dropna(subset=vital_cols, inplace=True)

    # 화씨 -> 섭씨 변환
    mask_fahrenheit = df_matched['temperature'] > 50
    df_matched.loc[mask_fahrenheit, 'temperature'] = (df_matched.loc[mask_fahrenheit, 'temperature'] - 32) * 5.0/9.0

    # 이상치 제거
    clean_mask = (
        (df_matched['heartrate'] > 20) & (df_matched['heartrate'] < 300) &
        (df_matched['o2sat'] > 0) & (df_matched['o2sat'] <= 100) &
        (df_matched['sbp'] > 30) & (df_matched['sbp'] < 300) &
        (df_matched['dbp'] > 10) & (df_matched['dbp'] < 200) &
        (df_matched['temperature'] > 10) & (df_matched['temperature'] < 45) &
        (df_matched['resprate'] > 0) & (df_matched['resprate'] < 100)
    )
    df_cleaned = df_matched[clean_mask].copy()

    # 환자 정보 및 라벨 병합
    df_final = pd.merge(df_cleaned, df_patients, on='subject_id', how='left')
    df_final = df_final[df_final['gender'].isin(['M', 'F'])]
    df_final['gender'] = df_final['gender'].apply(lambda x: 1 if x == 'M' else 0)
    df_final.dropna(subset=['anchor_age'], inplace=True)

    # 나이 정규화
    age_mean = df_final['anchor_age'].mean()
    age_std = df_final['anchor_age'].std()
    df_final['anchor_age'] = (df_final['anchor_age'] - age_mean) / age_std

    # 라벨 병합
    df_final = pd.merge(df_final, df_chexpert, on=['subject_id', 'study_id'], how='left')
    target_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']
    for col in target_diseases:
        if col in df_final.columns:
            df_final[col] = df_final[col].fillna(0).replace(-1, 1)

    # 이미지 경로 생성
    def get_img_path(row):
        subj = str(row['subject_id'])
        study = str(row['study_id'])
        dicom = str(row['dicom_id'])
        return f"p{subj[:2]}/p{subj}/s{study}/{dicom}.jpg"

    df_final['path'] = df_final.apply(get_img_path, axis=1)

    # ==============================================================================
    # 5. 최종 저장 (WBC, BNP 포함)
    # ==============================================================================
    final_cols = ['subject_id', 'stay_id', 'study_id', 'path', 'gender', 'anchor_age',
                  'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'acuity',
                  'WBC', 'BNP'] + target_diseases  # <--- [중요] 여기에 추가됨!

    available_cols = [c for c in final_cols if c in df_final.columns]
    df_export = df_final[available_cols]

    save_path = SAVE_DIR + 'master_dataset.csv'
    df_export.to_csv(save_path, index=False)

    print("\n" + "="*50)
    print(f"🎉 [완료] Master Dataset 생성 성공! (Lab 데이터 포함)")
    print(f"📂 저장 경로: {save_path}")
    print(f"📊 최종 데이터 개수: {len(df_export)}개")
    print("="*50)

except Exception as e:
    print(f"\n❌ 에러 발생: {e}")

데이터셋 중 누락된 이미지들도 있기 때문에 누락된 이미지 행들을 지웁니다.

In [ ]:
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

# ==============================================================================
# 1. 설정
# ==============================================================================
BASE_DIR = '/content/drive/MyDrive/REM_project/github/dataset/split_data/'
INPUT_CSV = BASE_DIR + 'master_dataset.csv'
OUTPUT_CSV = BASE_DIR + 'master_dataset_verified.csv'

# 🚨 실제 이미지가 있는 최상위 폴더
IMG_ROOT = '/content/drive/MyDrive/mimic_cxr_data/raw'

print("🚀 [경로 수정 및 검증] 시작...")

# ==============================================================================
# 2. 데이터 로드
# ==============================================================================
df = pd.read_csv(INPUT_CSV)
print(f"📂 원본 데이터 로드: {len(df)}개")

# ==============================================================================
# 2.5. [핵심 수정] 경로 문자열 고치기 (files/ 제거)
# ==============================================================================
# 설명: CSV의 'files/p10/...' 부분을 'p10/...'로 바꿉니다.
# 이렇게 해야 IMG_ROOT와 합쳤을 때 경로가 딱 맞습니다.
print("🔧 경로 문자열 수정 중 ('files/' 제거)...")
df['path'] = df['path'].str.replace('files/', '', regex=False)

# 확인용 출력
print(f"   👉 수정 후 경로 예시: {df['path'].iloc[0]}")
# 예상 출력: p10/p10000032/... (files/가 없어야 함)

# ==============================================================================
# 3. 검증 함수
# ==============================================================================
def check_file_exists(row_path):
    # 이제 row_path에는 'files/'가 없으므로 바로 합치면 됩니다.
    full_path = os.path.join(IMG_ROOT, str(row_path))
    return os.path.exists(full_path)

# ==============================================================================
# 4. 고속 병렬 검증
# ==============================================================================
print("   🕵️‍♂️ 파일 시스템 스캔 중...")
paths = df['path'].tolist()

# 구글 드라이브 안정성을 위해 workers=16 권장
with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(check_file_exists, paths), total=len(paths), desc="Checking Files"))

# ==============================================================================
# 5. 저장
# ==============================================================================
df_verified = df[results].copy()
deleted_count = len(df) - len(df_verified)

print("-" * 50)
print(f"✅ 검증 완료!")
print(f"   - 원본 개수: {len(df)}")
print(f"   - 생존 개수: {len(df_verified)}")
print(f"   - 👻 삭제된 유령 데이터: {deleted_count}개")
print("-" * 50)

if len(df_verified) > 0:
    os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
    df_verified.to_csv(OUTPUT_CSV, index=False)
    print(f"💾 저장 완료: {OUTPUT_CSV}")
    print("👉 저장된 파일은 경로가 'p10/...' 형태로 깔끔하게 정리되었습니다.")
else:
    print("🚨 여전히 0개입니다. 경로를 print해서 다시 확인해보세요.")

논문에 넣기 위한 데이터 통계 표를 뽑습니다.

In [ ]:
from google.colab import drive
import os

# Attempt to unmount first, in case of a problematic previous mount state
try:
  drive.flush_and_unmount()
  print("Google Drive unmounted successfully for remount.")
except Exception:
  print("Google Drive was not mounted or could not be unmounted. Proceeding with mount.")

# Explicitly clear the mount point to avoid 'Mountpoint must not already contain files' error
if os.path.exists('/content/drive'):
    print("Clearing existing /content/drive directory...")
    !rm -rf /content/drive/*

drive.mount('/content/drive', force_remount=True)


import pandas as pd
import numpy as np

# ==============================================================================
# 1. 데이터 로드 (경로 확인 필수!)
# ==============================================================================
# 01번 파일에서 생성된 최종 데이터셋 경로입니다.
path = '/content/drive/MyDrive/REM_project/github/dataset/split_data/master_dataset_verified.csv'

try:
    df = pd.read_csv(path)
    print(f"✅ 데이터 로드 성공! 총 {len(df)}명의 환자 데이터가 있습니다.")
except FileNotFoundError:
    print("❌ 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

# ==============================================================================
# 2. 질환별 통계 (Disease Prevalence)
# ==============================================================================
print("\n" + "="*40)
print("🏥 질환별 발생 빈도 (Prevalence)")
print("="*40)

target_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion', 'Pneumonia']

summary_disease = []
for disease in target_diseases:
    if disease in df.columns:
        count = df[disease].sum()
        ratio = (count / len(df)) * 100
        summary_disease.append([disease, int(count), f"{ratio:.1f}%"])

df_disease_stats = pd.DataFrame(summary_disease, columns=['Disease', 'Count', 'Percentage'])
print(df_disease_stats)

# ==============================================================================
# 3. 바이탈 및 인구통계 (Clinical Characteristics)
# ==============================================================================
print("\n" + "="*40)
print("🌡️ 임상적 특징 (Mean ± Std)")
print("="*40)

# 분석할 컬럼 목록
clinical_cols = ['anchor_age', 'gender', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'WBC', 'BNP']

stats_list = []

for col in clinical_cols:
    if col in df.columns:
        # 데이터 가져오기
        data = df[col]

        # [주의] WBC, BNP는 결측치가 -1로 채워져 있으므로, -1을 제외하고 통계 계산
        if col in ['WBC', 'BNP']:
            data = data[data != -1]
            missing_rate = (1 - len(data)/len(df)) * 100
            note = f"(결측 {missing_rate:.1f}%)"
        else:
            note = ""

        # 통계 계산
        mean_val = data.mean()
        std_val = data.std()
        min_val = data.min()
        max_val = data.max()

        # 성별(Gender)은 평균 대신 비율로 표시
        if col == 'gender':
            male_count = data.sum()
            male_ratio = data.mean() * 100
            stats_list.append([col, f"Male: {int(male_count)} ({male_ratio:.1f}%)", "-", "-", "-"])

        # 나이(Age)는 정규화(Z-score) 되어 있음을 명시
        elif col == 'anchor_age':
            stats_list.append([col, f"{mean_val:.2f} ± {std_val:.2f}", f"{min_val:.2f}", f"{max_val:.2f}", "Normalized"])

        else:
            stats_list.append([col, f"{mean_val:.1f} ± {std_val:.1f}", f"{min_val:.1f}", f"{max_val:.1f}", note])

# 데이터프레임으로 이쁘게 출력
df_stats = pd.DataFrame(stats_list, columns=['Feature', 'Mean ± Std (or Count)', 'Min', 'Max', 'Note'])
print(df_stats)